In [31]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")

# 변수명을 항상 sc로 설정할 것
sc = SparkContext(conf=conf)
sc

<SparkContext master=local appName=restaurant-review-average>

In [32]:
filepath = "/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv"

In [33]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect() # collect는 Action! Action은 데이터가 다 보임

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [34]:
# header 제거하기
header = lines.first()
header

'id,item,cateogry,reviews,'

In [35]:
rows = lines.filter(lambda row : row != header)
rows

PythonRDD[3] at RDD at PythonRDD.scala:53

In [36]:
rows.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [37]:
# 각 행을 파싱하기 위한 Task (함수)
def parse(row):
    fields = row.split(",")
    
    # 카테고리 얻어오기
    category = fields[2]
    
    # 리뷰 개수 얻어오기
    review_count = fields[3]
    review_count = int(review_count)
    
    return category, review_count

In [38]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [39]:
# RDD 내의 모든 원소(row)에 대해 parse 함수를 적용하고 추출 --> map
category_reviews = rows.map(parse)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [40]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [41]:
# 키는 변경하지 않고 Value만 변경하기 -> MapValues

# - 각 카테고리 별 ( 리뷰의 개수, 종류의 개수 )

category_reviews_count = category_reviews.mapValues(lambda review_count : (review_count, 1))
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [42]:
reduce = category_reviews_count.reduceByKey(lambda x,y : (x[0]+y[0], x[1]+y[1]))
reduce.collect()


[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [44]:
average = reduce.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [45]:
sc.stop()